## 【問題1】SimpleRNNのフォワードプロパゲーション実装

In [1]:
import numpy as np

In [2]:
class SimpleRNN:
    
    """
    シンプルなRNN
    
    Parameters
    ----------
    lr : 学習率
    
    """
    
    def __init__(self, lr, 
                 #n_features, n_nodes, 
                 #initializer, optimizer
                ):
        
        #self.optimizer = optimizer
        
        # initializer のメソッドを実行し初期値を生成、以後更新
        #self.w_x = initializer.w_x(n_features, n_nodes)
        #self.w_h = initializer.w_h(n_nodes, n_nodes)
        #self.b = initializer.B(n_nodes)
        
        
        # 今回はコンストラクタで直接初期値を生成する
        self.w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
        self.w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
        self.b = np.array([1, 1, 1, 1]) # (n_nodes,)
        
        self.h  =None
        self.h_all = None
        self.x = None
        
        self.lr = lr
        self.dh = None
        

    def forward(self, x, h):
        """
        フォワード
        
        Parameters
        ----------
        x : 次の形のndarray, shape (batch_size, n_sequences, n_features)
            入力
            
        h :  次の形のndarray, shape (batch_size, n_nodes)
            前の時刻の状態h
          
        
        Returns
        ----------
        h : 次の形のndarray, shape (batch_size, n_nodes)
            更新された状態h
            
        """
        
        # 入力Xをコントラクタに保存、backwardで使う
        self.x = x
        
        # コンストラクタの重み・バイアスの値をロード
        w_h, w_x, b = self.w_h, self.w_x, self.b
        
        # 系列長を取得
        n_sequences = x.shape[1] # 3
        
        # 最初にコンストラクタに h を保存、以後時刻 t ごとに更新 
        self.h = h
        
        # 系列長分のhを収める箱を生成
        self.h_all = np.zeros((x.shape[0], x.shape[1], w_x.shape[1])) # (batch_size, n_sequences, n_nodes)
        
        # 時刻t ごとの処理
        for t in range(n_sequences):
        
            h1 = x[:, t, :] @ w_x
            #print(h1.shape)
            
            h2 = self.h @ w_h
            #print(h2.shape)

            h3 = h1+ h2
            #print(h3)

            a = h3 + b
            #print(a)
            
            self.h = np.tanh(a)
            
            # 上式をまとめると、
            # self.h = np.tanh(x[:, t, :] @ w_x + self.h @ w_h + b)
            
            self.h_all[:, t, :] += self.h # 逆伝播で使う
            #print(self.h_all)
        
        
        # 最後の h のみ出力
        return self.h
    
    
    def backward(self, dh):
        
        """
        バックワード
        
        Parameters
        ----------
        dh : 次の形のndarray, shape (batch_size, n_nodes)
            後ろから流れてきたhの勾配
            
        Returns
        ----------
        dh : 次の形のndarray, shape (n_features, n_nodes)
            前に流すhの勾配
            
        dx : 次の形のndarray, shape (n_nodes, n_nodes)
            前に流すxの勾配
        
        """
        
        lr = self.lr # 学習率
        
        # 入力の系列長を取得
        n_sequences = self.x.shape[1]
        
        # 最初にコンストラクタに dh を保存、以後時刻 t ごとに順伝播時と逆順に更新 
        #self.dh = dh
        
        # tanh関数の微分
        da = self.h * (1 - self.h**2)    
        
        # dx を入れる箱
        dx = np.zeros((x.shape))
        
        # 順伝播時とは逆順のループ処理を行う
        for t in reversed(range(n_sequences)):
            
            # バイアスの勾配
            db  = da
            # w_h の勾配
            dw_h = self.h_all[:, t, :].T @ da
            
            # w_x の勾配
            dw_x = self.x[:, t, :].T @ da
            
            # h, x の勾配を、t回（系列長分）更新
            dh += da @ self.w_h.T
            dx[:, t, :] += da @ self.w_x.T
            
            # パラメータの更新 (t回) 
            self.b = self.b.astype(np.float) - lr * db
            self.w_h -= lr * dw_h
            self.w_x -= lr * dw_x
        
        # 更新
        #self.optimizer.update(self)
        
        return dh, dx

## 【問題2】小さな配列でのフォワードプロパゲーションの実験

In [3]:
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100 # (batch_size, n_sequences, n_features)

w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)

batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2

n_nodes = w_x.shape[1] # 4

h = np.zeros((batch_size, n_nodes)) # (batch_size, n_nodes)
b = np.array([1, 1, 1, 1]) # (n_nodes,)


# 入力の型, 系列 0 番目の型
x.shape, x[:, 0, :].shape

((1, 3, 2), (1, 2))

In [4]:
# インスタンス生成

rnn = SimpleRNN(lr=0.01)

In [5]:
# 順伝播

rnn.forward(x, h)

array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]])

In [6]:
h = np.array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]]) # (batch_size, n_nodes)

## 【問題3】（アドバンス課題）バックプロパゲーションの実装

### 逆伝播

In [7]:
dh_prev, dx = rnn.backward(h)
dh_prev, dx

(array([[0.90682948, 0.96128325, 1.00644579, 1.0608213 ]]),
 array([[[0.03932372, 0.05777536],
         [0.03937823, 0.05785713],
         [0.03946   , 0.05796616]]]))

In [8]:
dh_prev.shape, dx.shape

((1, 4), (1, 3, 2))

In [9]:
# 欲しい型
# dh_prev = (1, 4) = (batch_size, n_nodes)
# dx = (1, 3, 2) = (batch_size, n_sequences, n_features)

### 更新された各重み

In [10]:
rnn.w_x, rnn.w_h

(array([[0.00982444, 0.02983784, 0.04985122, 0.06986262],
        [0.02973667, 0.04975676, 0.06977683, 0.07979393]]),
 array([[0.0031269 , 0.02365141, 0.04417511, 0.06462146],
        [0.01299327, 0.03352797, 0.05406185, 0.07451688],
        [0.02287294, 0.04341683, 0.06395988, 0.07442272],
        [0.03277831, 0.05332942, 0.07387968, 0.09434867]]))

In [11]:
rnn.w_x.shape, rnn.w_h.shape

((2, 4), (4, 4))

In [12]:
# 欲しい型
# w_x = (2, 4) = (n_features, n_nodes)
# d_h = (4, 4) = (n_nodes, n_nodes)